In [ ]:
'''
Commander Lambda has tasked you to help the scientists increase fuel creation efficiency by predicting the end state of a given ore sample. You have carefully studied the different structures that the ore
can take and which transitions it undergoes. It appears that, while random, the probability of each structure transforming is fixed. That is, each time the ore is in 1 state, it has the same probabilities
of entering the next state (which might be the same state).  You have recorded the observed transitions in a matrix. The others in the lab have hypothesized more exotic forms that the ore can become, but
you haven't seen all of them.

Write a function solution(m) that takes an array of array of nonnegative ints representing how many times that state has gone to the next state and return an array of ints for each terminal state giving
the exact probabilities of each terminal state, represented as the numerator for each state, then the denominator for all of them at the end and in simplest form. The matrix is at most 10 by 10. It is
guaranteed that no matter which state the ore is in, there is a path from that state to a terminal state. That is, the processing will always eventually end in a stable state. The ore starts in state 0.
The denominator will fit within a signed 32-bit integer during the calculation, as long as the fraction is simplified regularly.

For example, consider the matrix m:
[
  [0,1,0,0,0,1],  # s0, the initial state, goes to s1 and s5 with equal probability
  [4,0,0,3,2,0],  # s1 can become s0, s3, or s4, but with different probabilities
  [0,0,0,0,0,0],  # s2 is terminal, and unreachable (never observed in practice)
  [0,0,0,0,0,0],  # s3 is terminal
  [0,0,0,0,0,0],  # s4 is terminal
  [0,0,0,0,0,0],  # s5 is terminal
]
So, we can consider different paths to terminal states, such as:
s0 -> s1 -> s3
s0 -> s1 -> s0 -> s1 -> s0 -> s1 -> s4
s0 -> s1 -> s0 -> s5
Tracing the probabilities of each, we find that
s2 has probability 0
s3 has probability 3/14
s4 has probability 1/7
s5 has probability 9/14
So, putting that together, and making a common denominator, gives an answer in the form of
[s2.numerator, s3.numerator, s4.numerator, s5.numerator, denominator] which is
[0, 3, 2, 9, 14].


Test cases
==========

Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Java cases --

Input:
Solution.solution({{0, 1, 0, 0, 0, 1}, {4, 0, 0, 3, 2, 0}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0}})
Output:
    [0, 3, 2, 9, 14]

Input:
Solution.solution({{0, 2, 1, 0, 0}, {0, 0, 0, 3, 4}, {0, 0, 0, 0, 0}, {0, 0, 0, 0,0}, {0, 0, 0, 0, 0}})
Output:
    [7, 6, 8, 21]

-- Python cases --

Input:
solution.solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])
Output:
    [7, 6, 8, 21]

Input:
solution.solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
Output:
    [0, 3, 2, 9, 14]

Use verify [file] to test your solution and see how it does.
:


'''

In [19]:

from fractions import Fraction
from math import gcd


def fraction(n, d=1):
    if n == 0:
        return 0
    return Fraction(n, d)


def identity(A):
    n = range(len(A))
    return [[1 if i == j else 0 for j in n] for i in n]


def subtract(A, B):
    if isinstance(A,list):
         return [ subtract(ra,rb) for ra,rb in zip(A,B) ]
    else:
         return A-B



def get_multiplied_matrix(A, B):
    return [[sum(a*b for a,b in zip(A_row, B_col)) for B_col in zip(*B)] for A_row in A]


def get_transpose_Matrix(m):
    return map(list,zip(*m))

def get_Matrix_Minor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

def get_Matrix_Deternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*get_Matrix_Deternminant(get_Matrix_Minor(m,0,c))
    return determinant

def get_inverse_matrix(m):
    determinant = get_Matrix_Deternminant(m)
    #special case for 2x2 matrix:
    if len(m) == 2:
        return [[m[1][1]/determinant, -1*m[0][1]/determinant],
                [-1*m[1][0]/determinant, m[0][0]/determinant]]

    #find matrix of cofactors
    cofactors = []
    for r in range(len(m)):
        cofactorRow = []
        for c in range(len(m)):
            minor = get_Matrix_Minor(m,r,c)
            cofactorRow.append(((-1)**(r+c)) * get_Matrix_Deternminant(minor))
        cofactors.append(cofactorRow)
    cofactors = get_transpose_Matrix(cofactors)
    for r in range(len(cofactors)):
        for c in range(len(cofactors)):
            cofactors[r][c] = cofactors[r][c]/determinant
    return cofactors



def get_lcm(a):
    for l, m in enumerate(a):
        if l==0:
            lcm = m
        else:
            lcm = int(lcm * m / gcd(lcm, m))
    return lcm


def get_MC_matrix(Sq):
    return get_lcm([x.denominator for x in Sq])


def solution(m):    

    sparse_matrix = []
    
    for row in m:
        sparse_matrix.append(not any(row))

    if sparse_matrix.count(True) == 1:
        return [1, 1]

    P=[[]]
    
    for current, val in enumerate(m):
        s=[]
        for next_c, prob in enumerate(val):
            if sparse_matrix[current] and current == next_c:
                s.append(1)
            else:
                s.append(fraction(prob, sum(m[current])))
        
        P.append(s)
    P = P[1:]
    
    

    Q=[[]]
    
    for i, check in enumerate(sparse_matrix):
        if not check:
            s=[]
            for j, bool_val in enumerate(sparse_matrix):
                if not bool_val:
                    s.append(P[i][j])
    #         if not is_terminal:
            Q.append(s)
    
    Q = Q[1:]
    
    

    R=[[]]
    
    for i, check in enumerate(sparse_matrix):
        if not check:
            s=[]
            for j, bool_val in enumerate(sparse_matrix):
                if bool_val:
                    s.append(P[i][j])
    #         if not is_terminal:
            R.append(s)
    
    R = R[1:]
    
    I = identity(Q)


    Sq = get_multiplied_matrix(get_inverse_matrix(subtract(I, Q)), R)[0]

    marcov_matrix = get_MC_matrix(Sq)

    return [int(x.numerator * marcov_matrix / x.denominator) for x in Sq] + [marcov_matrix]


In [20]:
solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])

[7, 6, 8, 21]

In [3]:
solution([[0, 1, 0, 0, 0, 1], [4, 0, 0, 3, 2, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])

[0, 3, 2, 9, 14]

In [17]:

from fractions import Fraction
from math import gcd


def fraction(n, d=1):
    if n == 0:
        return 0
    return Fraction(n, d)


def identity(A):
    n = range(len(A))
    return [[1 if i == j else 0 for j in n] for i in n]


def subtract(A, B):
    if isinstance(A,list):
         return [ subtract(ra,rb) for ra,rb in zip(A,B) ]
    else:
         return A-B



def get_multiplied_matrix(A, B):
    return [[sum(a*b for a,b in zip(A_row, B_col)) for B_col in zip(*B)] for A_row in A]


def get_transpose_Matrix(m):
    return map(list,zip(*m))

def get_Matrix_Minor(m,i,j):
    return [row[:j] + row[j+1:] for row in (m[:i]+m[i+1:])]

def get_Matrix_Deternminant(m):
    #base case for 2x2 matrix
    if len(m) == 2:
        return m[0][0]*m[1][1]-m[0][1]*m[1][0]

    determinant = 0
    for c in range(len(m)):
        determinant += ((-1)**c)*m[0][c]*get_Matrix_Deternminant(get_Matrix_Minor(m,0,c))
    return determinant

# def get_inverse_matrix(m):
#     determinant = get_Matrix_Deternminant(m)
#     #special case for 2x2 matrix:
#     if len(m) == 2:
#         return [[m[1][1]/determinant, -1*m[0][1]/determinant],
#                 [-1*m[1][0]/determinant, m[0][0]/determinant]]

#     #find matrix of cofactors
#     cofactors = []
#     for r in range(len(m)):
#         cofactorRow = []
#         for c in range(len(m)):
#             minor = get_Matrix_Minor(m,r,c)
#             cofactorRow.append(((-1)**(r+c)) * get_Matrix_Deternminant(minor))
#         cofactors.append(cofactorRow)
#     cofactors = get_transpose_Matrix(cofactors)
#     for r in range(len(cofactors)):
#         for c in range(len(cofactors)):
#             cofactors[r][c] = cofactors[r][c]/determinant
#     return cofactors


def get_inverse_matrix(a):
    b = identity(a)
    for d in xrange(len(a)):
        to1 = fraction(1, a[d][d])
        for j in xrange(len(a)):
            a[d][j] *= to1
            b[d][j] *= to1
        for i in range(len(a))[0:d] + range(len(a))[d + 1 :]:
            to0 = a[i][d]
            for j in xrange(len(a)):
                a[i][j] = a[i][j] - to0 * a[d][j]
                b[i][j] = b[i][j] - to0 * b[d][j]
    return b



def get_lcm(a):
    for l, m in enumerate(a):
        if l==0:
            lcm = m
        else:
            lcm = int(lcm * m / gcd(lcm, m))
    return lcm


def get_MC_matrix(Sq):
    return get_lcm([x.denominator for x in Sq])


def solution(m):    

    terminal = [not any(row) for row in m]

    if terminal.count(True) == 1:
        return [1, 1]

    p = [
        [
            1
            if terminal[state] and state == next_state
            else fraction(prob, sum(m[state]))
            for next_state, prob in enumerate(probs)
        ]
        for state, probs in enumerate(m)
    ]

    q = [
        [p[i][j] for j, is_terminal in enumerate(terminal) if not is_terminal]
        for i, is_terminal in enumerate(terminal)
        if not is_terminal
    ]

    r = [
        [p[i][j] for j, is_terminal in enumerate(terminal) if is_terminal]
        for i, is_terminal in enumerate(terminal)
        if not is_terminal
    ]

    # probabilities for starting in state 0
    b0 = get_multiplied_matrix(get_inverse_matrix(subtract(identity(q), q)), r)[0]

    common = get_lcm([x.denominator for x in b0])

    return [int(x.numerator * common / x.denominator) for x in b0] + [common]


In [18]:
solution([[0, 2, 1, 0, 0], [0, 0, 0, 3, 4], [0, 0, 0, 0, 0], [0, 0, 0, 0,0], [0, 0, 0, 0, 0]])

NameError: name 'xrange' is not defined